## Branch Network Mapping

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [2]:
EARTHRADIUS = 6371.0
def getDistanceByHaversine(loc1, loc2):
    dlon = loc1[1] - loc2[1]
    dlat = loc1[0] - loc2[0]
    a = (np.sin(dlat/2))**2 + loc1[2]  * loc2[2] * (np.sin(dlon/2.0))**2
    c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0-a))
    km = EARTHRADIUS * c
    return km

In [3]:
def matrix_com(matrix):
    dist_list = []
    for cols1 in range(matrix.shape[0]):
        M = np.array([matrix[cols1, 0], matrix[cols1, 1], matrix[cols1, 2]])
        for cols2 in range(cols1+1, matrix.shape[0]):
                N = np.array([matrix[cols2, 0], matrix[cols2, 1], matrix[cols2, 2]])
                dist = getDistanceByHaversine(M, N)
                dist_list.append(dist)
    return dist_list

In [5]:
def get_centermost_point(cluster):
    # Extracting geometric centre
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

### Aim

Assuming  Contrywide was to start with a blank canvas operating with 2 brands i.e. prime and value:
   
   . How many prime and value branches 
   
   . Where should the branches be located
   
   . What is the revenue opportunity for each branch

In [6]:
df = pd.read_csv('lrg.txt', sep='\t', header=None)
df.rename(columns={0: 'Post_Area', 1: 'Latitude', 2: 'Longitude', 3: 'Property_Type', 4: 'Property_Price'},
         inplace=True)
df.head()

,Post_Area,Latitude,Longitude,Property_Type,Property_Price
0,NG,53.026627,-1.202770,Value,126000
1,NG,53.026341,-1.208354,Value,160000
2,NG,53.026341,-1.208354,Value,120500
3,NG,53.025055,-1.221487,Value,235000
4,NG,53.025055,-1.221487,Value,135000


In [7]:
inst_df = pd.read_csv('Instructions.txt', sep='\t')
inst_df['Instructions'] = [round(i) for i in inst_df['Instructions']]
inst_df.head()

,Post_Area,Property_Type,Instructions
0,TW,Prime,18.0
1,HU,Value,125.0
2,DG,Value,79.0
3,BA,Prime,11.0
4,EC,Prime,18.0


In [8]:
df['Lat_Radian'] = df['Latitude']  * np.pi / 180.0
df['Long_Radian'] = df['Longitude']  * np.pi / 180.0
df['Cos_Lat_Rad'] = [np.cos(x) for x in df['Lat_Radian']]
df.head()

,Post_Area,Latitude,Longitude,Property_Type,Property_Price,Lat_Radian,Long_Radian,Cos_Lat_Rad
0,NG,53.026627,-1.202770,Value,126000,0.925489,-0.020992,0.601444
1,NG,53.026341,-1.208354,Value,160000,0.925484,-0.021090,0.601448
2,NG,53.026341,-1.208354,Value,120500,0.925484,-0.021090,0.601448
3,NG,53.025055,-1.221487,Value,235000,0.925462,-0.021319,0.601466
4,NG,53.025055,-1.221487,Value,135000,0.925462,-0.021319,0.601466


In [25]:
pa_df = df['Post_Area'].value_counts()
(pa_df.reset_index()).sort_values('Post_Area')[:15]

,index,Post_Area
104,TD,305
103,WC,319
102,EC,697
101,LD,715
100,HX,2631
99,HG,2748
98,HR,2945
97,SR,3148
96,SM,3469
95,UB,3502


#### Calculation to define max distance between 2 points in a prime and value cluster for each postcode area

In [26]:
dist_final_df = pd.DataFrame()
# for post_area in df['Post_Area'].unique():
for post_area in ['WD']:
    post_df = df[df['Post_Area'] == post_area]
    for branch_type in post_df['Property_Type'].unique():
        branch_df = post_df[post_df['Property_Type'] == branch_type]
        measure_list = []    
        cords = branch_df[['Lat_Radian', 'Long_Radian', 'Cos_Lat_Rad']].values
        dist_list = matrix_com(cords)
        dist_list = [ i for i in dist_list if i > 0]
        dist_threhold_1 = np.average(dist_list) / 10
        dist_threhold_2 = np.percentile(dist_list, 0.25)
        dist_threhold_3 = np.average(dist_list) / 5
        dist_threhold_4 = np.percentile(dist_list, 0.5)
        dist_threhold_5 = np.average(dist_list)
        measure_list.append((dist_threhold_1, dist_threhold_2, dist_threhold_3, 
                             dist_threhold_4, dist_threhold_5, post_area, branch_type))
        dist_df = pd.DataFrame(measure_list, columns = ['dist_threhold_1', 'dist_threhold_2', 
                                                       'dist_threhold_3', 'dist_threhold_4', 'dist_threhold_5', 
                                                       'Post_Area', 'Property_Type'])
        dist_final_df = pd.concat([dist_final_df,dist_df])

In [27]:
dist_final_df

,dist_threhold_1,dist_threhold_2,dist_threhold_3,dist_threhold_4,dist_threhold_5,Post_Area,Property_Type
0,0.702509,0.192051,1.405018,0.283269,7.025092,WD,Prime
0,0.582867,0.155321,1.165734,0.230514,5.828669,WD,Value


In [28]:
final_label_df = pd.DataFrame()
branch_final_df = pd.DataFrame()
final_rep_points = pd.DataFrame()

# for post_area in df['Post_Area'].unique():
for post_area in ['WD']:
    post_df = df[df['Post_Area'] == post_area]
    for branch_type in post_df['Property_Type'].unique():
        branch_df = post_df[post_df['Property_Type'] == branch_type]
        # No of avg instruction required to be successful in postcode area for branch type
        new_inst_df = inst_df[inst_df['Post_Area'] == post_area]
        new_inst_df = new_inst_df[new_inst_df['Property_Type'] == branch_type]
        
        new_dist_df = dist_final_df[dist_final_df['Post_Area'] == post_area]
        new_dist_df = new_dist_df[new_dist_df['Property_Type'] == branch_type]
        dist_threhold_1 = new_dist_df['dist_threhold_1'][0]
        dist_threhold_2 = new_dist_df['dist_threhold_2'][0]
        x = 0.40
        y = 0.07
        # Assuming a 50% conversion rate from instruction to exchange(x) and a 7% market share(y) for branch
        sample_size = (new_inst_df['Instructions'].values[0] * x) / y
        # Allowing a 20% wiggle room for sample size
        sample_size = sample_size - (0.2 * sample_size)
        'Sample size for {} branch is {} '.format(branch_type, sample_size)
        if branch_type == 'Prime':
            if post_area in ['SE', 'SW', 'NW', 'W', 'E', 'EC', 'WC','N']:
            # Epsilon to define London is smaller as 
            # London is predominately a prime region hence allowing for more prime branches
                dist_threhold = dist_threhold_2
            else:
                dist_threhold = dist_threhold_1
        else:
            dist_threhold = dist_threhold_2
        'Max distance for {} branch is {} '.format(branch_type, dist_threhold)      
        coords = branch_df.as_matrix(columns=['Lat_Radian', 'Long_Radian'])
        kms_per_radian = 6371.0088
        # Convert max distance to radians
        epsilon = dist_threhold / kms_per_radian
        ms = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(coords)
        cluster_labels = ms.labels_
        num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
        'Original Cluster Size for {} branch is {} '.format(branch_type, num_clusters)
        label_df = pd.DataFrame({ "label": cluster_labels})
        label_df = label_df[label_df["label"] <> -1]
        label_df['Count'] = label_df.groupby('label')['label'].transform('count')
        label_df.drop_duplicates(inplace=True)
        label_df.sort_values('Count', ascending=False, inplace=True)
        label_df_new = label_df[label_df['Count'] >= sample_size]
        rep_points = pd.DataFrame() 
        if len(label_df_new) > 0:
            clusters = pd.Series([coords[cluster_labels == n] for n in label_df_new['label'].unique()])
            centermost_points = clusters.map(get_centermost_point)    
            lats, lons = zip(*centermost_points)
            # Dataframe for cluster centres
            rep_points = pd.DataFrame({'lon':lons, 'lat':lats}) 
            rep_points.loc[:, 'Post_Area'] = post_area
            rep_points.loc[:, 'Branch_Type'] = branch_type
        # If there are no clusters for a postcode area, reduce cluster size 
        # by further 20% 
        else:
            label_df_new = label_df[label_df['Count'] >= sample_size - (0.2 * sample_size)]
            if len(label_df_new) > 0:
                clusters = pd.Series([coords[cluster_labels == n] for n in label_df_new['label'].unique()])
                centermost_points = clusters.map(get_centermost_point)    
                lats, lons = zip(*centermost_points)
                rep_points = pd.DataFrame({'lon':lons, 'lat':lats}) 
                rep_points.loc[:, 'Post_Area'] = post_area
                rep_points.loc[:, 'Branch_Type'] = branch_type
        label_df_new['Branch_Type'] = branch_type
        label_df_new['Sample_Size'] = sample_size
        label_df_new['Distance Threshold'] = dist_threhold
        final_label_df = pd.concat([final_label_df, label_df_new])
        final_rep_points = pd.concat([final_rep_points, rep_points])

'Sample size for Prime branch is 187.428571429 '

'Max distance for Prime branch is 0.702509225079 '

'Original Cluster Size for Prime branch is 26 '

C:\Users\adetola.alade\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\adetola.alade\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\adetola.alade\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

'Sample size for Value branch is 242.285714286 '

'Max distance for Value branch is 0.155321110824 '

'Original Cluster Size for Value branch is 306 '

Using this model, I would recommend 2 prime branches and 1 value branch in Watford

In [29]:
final_label_df
final_rep_points

,label,Count,Branch_Type,Sample_Size,Distance Threshold
281,7,620,Prime,187.428571,0.702509
0,0,276,Prime,187.428571,0.702509
1023,187,232,Value,242.285714,0.155321


Further analysis was then carried out jupyter notebooks to do the following:

. Map the centroids to the nearest high street or retail park within an acceptable distance. 

. Check out this correlates with the existing network and measure degree of overlap.

. When communicating this back to key stakesholders, I was able to use the transaction value and assumed market share to calculate revenue opportunity for a branch in a cluster.